# Dense Embedding 을 활용한 ODQA 시스템 만들기

### Requirements

In [26]:
# > /dev/null 2>&1 # execute command in silence
!pip install datasets==1.4.1 > /dev/null 2>&1 # execute command in silence
!pip install transformers==4.20.1 > /dev/null 2>&1
!pip install tqdm==4.41.1 > /dev/null 2>&1

In [27]:
import torch
import random
from pprint import pprint

## Dense Embedding 을 활용한 Open-domain Question Answering 시스템 만들기


5강에서 배운대로 dense embedding을 만드는 encoder 을 학습시키기

또는 학습된 encoder 파일을 가져와서 진행하기

In [28]:
# 1. 미리 학습해둔 encoder file 다운로드
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=11XrSCVLrzN41S0ELe4Yd-NFJ21GSWmNx' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=11XrSCVLrzN41S0ELe4Yd-NFJ21GSWmNx" -O encoders.tar.gz && rm -rf /tmp/cookies.txt
# google file download link - https://drive.google.com/file/d/11XrSCVLrzN41S0ELe4Yd-NFJ21GSWmNx/view?usp=sharing

# 2. .tar.gz file 압축해제 
!tar -xf ./encoders.tar.gz

--2022-07-25 11:19:05--  https://docs.google.com/uc?export=download&confirm=t&id=11XrSCVLrzN41S0ELe4Yd-NFJ21GSWmNx
Resolving docs.google.com (docs.google.com)... 142.251.10.138, 142.251.10.113, 142.251.10.101, ...
Connecting to docs.google.com (docs.google.com)|142.251.10.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8a1ab7demihrds9pttr8ui26vk13hdbc/1658747925000/10000090644219567406/*/11XrSCVLrzN41S0ELe4Yd-NFJ21GSWmNx?e=download&uuid=6494d3f7-524a-4c96-9db6-0036ca801f27 [following]
--2022-07-25 11:19:06--  https://doc-04-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8a1ab7demihrds9pttr8ui26vk13hdbc/1658747925000/10000090644219567406/*/11XrSCVLrzN41S0ELe4Yd-NFJ21GSWmNx?e=download&uuid=6494d3f7-524a-4c96-9db6-0036ca801f27
Resolving doc-04-3s-docs.googleusercontent.com (doc-04-3s-docs.googleusercontent.com)... 142.251.10.132, 

In [29]:
# google drive 에 올려둔 미리 학습해둔 인코더 불러오기
from transformers import BertModel, BertPreTrainedModel, BertConfig, AutoTokenizer

class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      
      pooled_output = outputs[1]

      return pooled_output

# TODO: load model from path
p_encoder = BertEncoder.from_pretrained("./p_encoder").to('cuda')
q_encoder = BertEncoder.from_pretrained("./q_encoder").to('cuda')

model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

## Dense retriever 가져오기

In [30]:
# klue data가 포함된 wiki documents 가져오기
# google file download link - https://drive.google.com/file/d/1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd/view?usp=sharing
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd" -O wikipedia_documents.json && rm -rf /tmp/cookies.txt

--2022-07-25 11:19:52--  https://docs.google.com/uc?export=download&confirm=t&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd
Resolving docs.google.com (docs.google.com)... 74.125.200.102, 74.125.200.139, 74.125.200.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.200.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ce5ukj8gdlbu34em92e06mjsicg73qfu/1658747925000/10000090644219567406/*/1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd?e=download&uuid=3a876881-9043-431c-8e2d-d99626dc91f8 [following]
--2022-07-25 11:19:53--  https://doc-08-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ce5ukj8gdlbu34em92e06mjsicg73qfu/1658747925000/10000090644219567406/*/1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd?e=download&uuid=3a876881-9043-431c-8e2d-d99626dc91f8
Resolving doc-08-3s-docs.googleusercontent.com (doc-08-3s-docs.googleusercontent.com)... 142.251.10.132, 

In [31]:
# First load wikipedia dump
import json

dump_path = 'wikipedia_documents.json' 
with open(dump_path, 'r') as f:
    wiki = json.load(f)

corpus = [document['text'] for document_id, document in wiki.items()]

사전에 학습한 passage encoder, question encoder을 이용해 dense embedding 생성

생성된 embedding에 dot product를 수행 => Document들의 similarity ranking을 구함

Top-5개의 passage를 retrieve 하고 ground truth와 비교하기

In [32]:
from tqdm.auto import tqdm

# takes about 40mins ~ 1hr
# you can use below cell to download embedding
with torch.no_grad():
    p_encoder.eval()
    p_embs = []
    for p in tqdm(corpus):
        p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
        p_emb = p_encoder(**p).to('cpu').numpy()
        p_embs.append(p_emb)
    p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)

In [33]:
# download p_embs
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1uQ-k6lv86KXK0kRgsJw5qvFQg93s6GeI' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1uQ-k6lv86KXK0kRgsJw5qvFQg93s6GeI" -O wiki_p_embs.pt && rm -rf /tmp/cookies.txt
# google file download link - https://drive.google.com/file/d/1uQ-k6lv86KXK0kRgsJw5qvFQg93s6GeI/view?usp=sharing

p_embs = torch.load("wiki_p_embs.pt")

--2022-07-25 11:19:57--  https://docs.google.com/uc?export=download&confirm=t&id=1uQ-k6lv86KXK0kRgsJw5qvFQg93s6GeI
Resolving docs.google.com (docs.google.com)... 74.125.130.100, 74.125.130.102, 74.125.130.113, ...
Connecting to docs.google.com (docs.google.com)|74.125.130.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/t73d1dlru2ejrd4jpjp0j93evb0nvbjf/1658747925000/10000090644219567406/*/1uQ-k6lv86KXK0kRgsJw5qvFQg93s6GeI?e=download&uuid=14dc8b10-7359-401d-a990-490ac8b8ed65 [following]
--2022-07-25 11:19:58--  https://doc-10-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/t73d1dlru2ejrd4jpjp0j93evb0nvbjf/1658747925000/10000090644219567406/*/1uQ-k6lv86KXK0kRgsJw5qvFQg93s6GeI?e=download&uuid=14dc8b10-7359-401d-a990-490ac8b8ed65
Resolving doc-10-3s-docs.googleusercontent.com (doc-10-3s-docs.googleusercontent.com)... 142.251.10.132, 

retrieval 과정을 함수화 하기

In [34]:
def to_cuda(batch):
  return tuple(t.cuda() for t in batch)

In [35]:
def get_relevant_doc(p_encoder, q_encoder, query, k=1):
    with torch.no_grad():
        q_encoder.eval()

        q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
        q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

    # TODO: get dot product scores by using torch.matmul
    dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))

    # TODO: obtain rank (use torch.argsort)
    rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
    
    return dot_prod_scores.squeeze(), rank[:k]

In [36]:
""" 원하는 질문을 입력해보기 """
query = input("Enter any question: ") # "대한민국의 대통령은 누구인가?"
_, doc_id = get_relevant_doc(p_encoder, q_encoder, query, k=1)

""" 결과 확인 """

print("{} {} {}".format('*'*20, 'Result','*'*20))
print("[Search query]\n", query, "\n")
print(f"[Relevant Doc ID(Top 1 passage)]: {doc_id.item()}")
print(corpus[doc_id.item()])

Enter any question: 한민국의 대통령은 누구인가?
******************** Result ********************
[Search query]
 한민국의 대통령은 누구인가? 

[Relevant Doc ID(Top 1 passage)]: 36036
다른 한편 1912년 1월 1일부터 같은 해 1912년 4월 1일까지 쑨원(孫文)이 중화민국 대통령에 재임하였고 1912년 3월 10일부터 같은 해 1912년 4월 1일까지는 당시 중화민국 부통령 위안스카이(袁世凱)가 중화민국 대통령 권한대행을 지내다가 1912년 4월 1일부터 이듬해 1913년 10월 10일까지 중화민국 대통령을 지내었으며 이듬해 1913년 10월 10일부터 1915년 12월 12일까지 위안스카이가 중화민국 대총통을 지내다가 1915년 12월 12일부터 이듬해 1916년 3월 22일까지는 중화민국 체제를 폐지하고 중화제국(中華帝國)을 선포하여 중화제국 황제를 지내다가 1916년 3월 22일에 중화민국 국민당 잔당 측의 공습으로 중화제국이 붕괴되고 중화민국이 공화정 복고되어 1916년 3월 22일부터 같은 해 1916년 6월 7일까지 리위안훙(黎元洪)이 중화민국 국가원수 권한대행을 지내다가 1916년 6월 7일부터 이듬해 1917년 7월 14일까지 중화민국 대총통에 재임하였다.


In [37]:
""" 상위 5개를 추출하여 점수 확인 """
dot_prod_scores, rank = get_relevant_doc(p_encoder, q_encoder, query, k=5)

for i in range(5):
    print(rank[i])
    print("Top-%d passage with score %.4f" % (i+1, dot_prod_scores.squeeze()[rank[i]]))
    print(corpus[rank[i]])

tensor(36036)
Top-1 passage with score 40.3120
다른 한편 1912년 1월 1일부터 같은 해 1912년 4월 1일까지 쑨원(孫文)이 중화민국 대통령에 재임하였고 1912년 3월 10일부터 같은 해 1912년 4월 1일까지는 당시 중화민국 부통령 위안스카이(袁世凱)가 중화민국 대통령 권한대행을 지내다가 1912년 4월 1일부터 이듬해 1913년 10월 10일까지 중화민국 대통령을 지내었으며 이듬해 1913년 10월 10일부터 1915년 12월 12일까지 위안스카이가 중화민국 대총통을 지내다가 1915년 12월 12일부터 이듬해 1916년 3월 22일까지는 중화민국 체제를 폐지하고 중화제국(中華帝國)을 선포하여 중화제국 황제를 지내다가 1916년 3월 22일에 중화민국 국민당 잔당 측의 공습으로 중화제국이 붕괴되고 중화민국이 공화정 복고되어 1916년 3월 22일부터 같은 해 1916년 6월 7일까지 리위안훙(黎元洪)이 중화민국 국가원수 권한대행을 지내다가 1916년 6월 7일부터 이듬해 1917년 7월 14일까지 중화민국 대총통에 재임하였다.
tensor(57917)
Top-2 passage with score 40.0742
중공 역사상 최초의 본격적인 정식 주미 중공 대사 출신인데 1971년 둥비우(董必武) 당시 중화인민공화국 국가부원수 겸 국가원수 권한대행·저우언라이(周恩來) 당시 중화인민공화국 총리·린뱌오(林彪) 당시 중화인민공화국 부총리 겸 총리 권한대행 등을 비롯한 중화인민공화국 측과 리처드 닉슨(Richard Nixon) 당시 미국 대통령·헨리 키신저(Henry Kissinger) 당시 미국 국무부 국가안보담당보좌관 등을 비롯한 미국 측 양국 간의 "핑퐁 외교"가 수립된 후 중화인민공화국에서 소위 탁구를 비롯한 스포츠가 의외로 점차 개방되었으며 훗날 차이는 1974년 9월 2일을 기하여 이집트 주재 중공 대사 직책에서 퇴직 후 1979년 3월 29일부터 1983년 1월 8일까지 초대 미국 주재 중공 대사 직책을 4년

## 훈련된 MRC 모델 가져오기


In [38]:
import torch
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer
)

In [39]:
model_name = 'sangrimlee/bert-base-multilingual-cased-korquad'
mrc_model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [40]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True
)

In [41]:
mrc_model = mrc_model.eval()

In [42]:
def get_answer_from_context(context, question, model, tokenizer):
    # TODO: get encoded dictionary from question - use tokenizer.encode_plus
    encoded_dict = tokenizer.encode_plus(  
        question,
        context,
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    non_padded_ids = encoded_dict["input_ids"][: encoded_dict["input_ids"].index(tokenizer.pad_token_id)]
    full_text = tokenizer.decode(non_padded_ids)
    inputs = {
    'input_ids': torch.tensor([encoded_dict['input_ids']], dtype=torch.long),
    'attention_mask': torch.tensor([encoded_dict['attention_mask']], dtype=torch.long),
    'token_type_ids': torch.tensor([encoded_dict['token_type_ids']], dtype=torch.long)
    }

    outputs = model(**inputs)
    start, end = torch.max(outputs.start_logits, axis=1).indices.item(), torch.max(outputs.end_logits, axis=1).indices.item()
    if start == 0 and end == 0:
        answer = "This is not answerable"
    else:
        answer = tokenizer.decode(encoded_dict['input_ids'][start:end+1])
    return answer

In [43]:
context = corpus[doc_id.item()]
answer = get_answer_from_context(context, query, mrc_model, tokenizer)
print("{} {} {}".format('*'*20, 'Result','*'*20))
print("[Search query]\n", query, "\n")
print(f"[Relevant Doc ID(Top 1 passage)]: {doc_id.item()}")
pprint(corpus[doc_id.item()], compact=True)
print(f"[Answer Prediction from the model]: {answer}")

******************** Result ********************
[Search query]
 한민국의 대통령은 누구인가? 

[Relevant Doc ID(Top 1 passage)]: 36036
('다른 한편 1912년 1월 1일부터 같은 해 1912년 4월 1일까지 쑨원(孫文)이 중화민국 대통령에 재임하였고 1912년 3월 '
 '10일부터 같은 해 1912년 4월 1일까지는 당시 중화민국 부통령 위안스카이(袁世凱)가 중화민국 대통령 권한대행을 지내다가 1912년 '
 '4월 1일부터 이듬해 1913년 10월 10일까지 중화민국 대통령을 지내었으며 이듬해 1913년 10월 10일부터 1915년 12월 '
 '12일까지 위안스카이가 중화민국 대총통을 지내다가 1915년 12월 12일부터 이듬해 1916년 3월 22일까지는 중화민국 체제를 '
 '폐지하고 중화제국(中華帝國)을 선포하여 중화제국 황제를 지내다가 1916년 3월 22일에 중화민국 국민당 잔당 측의 공습으로 중화제국이 '
 '붕괴되고 중화민국이 공화정 복고되어 1916년 3월 22일부터 같은 해 1916년 6월 7일까지 리위안훙(黎元洪)이 중화민국 국가원수 '
 '권한대행을 지내다가 1916년 6월 7일부터 이듬해 1917년 7월 14일까지 중화민국 대총통에 재임하였다.')
[Answer Prediction from the model]: [UNK]


## 통합해서 ODQA 시스템 구축

In [44]:
def open_domain_qa(query, corpus, p_encoder, q_encoder, mrc_model, tokenizer, k=1):
    # 1. Retrieve k relevant docs by usign sparse matrix
    _, doc_id = get_relevant_doc(p_encoder, q_encoder, query, k=1)
    context = corpus[doc_id.item()]

    # 2. Predict answer from given doc by using MRC model
    answer = get_answer_from_context(context, query, mrc_model, tokenizer)
    print("{} {} {}".format('*'*20, 'Result','*'*20))
    print("[Search query]\n", query, "\n")
    print(f"[Relevant Doc ID(Top 1 passage)]: {doc_id.item()}")
    pprint(corpus[doc_id.item()], compact=True)
    print(f"[Answer Prediction from the model]: {answer}")

In [45]:
query = input("Enter any question: ") # "대한민국의 대통령은 누구인가?"
open_domain_qa(query, corpus, p_encoder, q_encoder, mrc_model, tokenizer, k=1)

Enter any question: 한민국의 대통령은 누구인가?
******************** Result ********************
[Search query]
 한민국의 대통령은 누구인가? 

[Relevant Doc ID(Top 1 passage)]: 36036
('다른 한편 1912년 1월 1일부터 같은 해 1912년 4월 1일까지 쑨원(孫文)이 중화민국 대통령에 재임하였고 1912년 3월 '
 '10일부터 같은 해 1912년 4월 1일까지는 당시 중화민국 부통령 위안스카이(袁世凱)가 중화민국 대통령 권한대행을 지내다가 1912년 '
 '4월 1일부터 이듬해 1913년 10월 10일까지 중화민국 대통령을 지내었으며 이듬해 1913년 10월 10일부터 1915년 12월 '
 '12일까지 위안스카이가 중화민국 대총통을 지내다가 1915년 12월 12일부터 이듬해 1916년 3월 22일까지는 중화민국 체제를 '
 '폐지하고 중화제국(中華帝國)을 선포하여 중화제국 황제를 지내다가 1916년 3월 22일에 중화민국 국민당 잔당 측의 공습으로 중화제국이 '
 '붕괴되고 중화민국이 공화정 복고되어 1916년 3월 22일부터 같은 해 1916년 6월 7일까지 리위안훙(黎元洪)이 중화민국 국가원수 '
 '권한대행을 지내다가 1916년 6월 7일부터 이듬해 1917년 7월 14일까지 중화민국 대총통에 재임하였다.')
[Answer Prediction from the model]: [UNK]
